In [ ]:
# ============================================
# CÉLULA 1: ARQUITETURA PRINCIPAL UNIFICADA
# ============================================
"""
ARQUITETURA UNIFICADA - 4 CAMADAS INTEGRADAS
┌─────────────────────────────────────────────────────────────┐
│                    CAMADA DE ORQUESTRAÇÃO                    │
│  ┌──────────────────────────────────────────────────────┐   │
│  │         Sistema Jurídico Unificado (Orchestrator)    │   │
│  │  • Coordena todos os componentes                     │   │
│  │  • Gerenciamento de fluxos de trabalho               │   │
│  │  • Balanceamento de carga entre MCPs                 │   │
│  └──────────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────────┘
                              │
┌─────────────────────────────────────────────────────────────┐
│                    CAMADA DE SERVIÇOS MCP                   │
│  ┌─────────────┐  ┌──────────────┐  ┌──────────────────┐   │
│  │ PageIndex   │  │   ChatIndex  │  │    Docling       │   │
│  │    MCP      │  │      MCP     │  │      MCP         │   │
│  │ (Árvores)   │  │ (Conversas)  │  │ (Extração)       │   │
│  └─────────────┘  └──────────────┘  └──────────────────┘   │
│  • Raciocínio     • Memória de      • OCR Avançado         │
│    em árvore        longa conversa  • Estrutura preservada │
│  • Busca sem        • B+ Trees      • Multi-modal          │
│    vetores        • Tópicos         • Tabelas/Imagens      │
└─────────────────────────────────────────────────────────────┘
                              │
┌─────────────────────────────────────────────────────────────┐
│                    CAMADA DE PROCESSAMENTO                  │
│  ┌──────────────────────────────────────────────────────┐   │
│  │          Pipeline Inteligente de Documentos          │   │
│  │  1. Extração Docling → 2. Indexação PageIndex →      │   │
│  │  3. Chunking Semântico → 4. ChatIndex Storage →      │   │
│  │  5. MCP Integration → 6. Auditoria                   │   │
│  └──────────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────────┘
                              │
┌─────────────────────────────────────────────────────────────┐
│                 CAMADA DE PERSISTÊNCIA UNIFICADA            │
│                    GOOGLE DRIVE ESTRUTURADO                 │
│  ┌──────────────────────────────────────────────────────┐   │
│  │  Juridico_Unificado/                                 │
│  │  ├── 00_MCP_Servers/     # Configurações MCP         │
│  │  ├── 01_PageIndex/       # Árvores de documentos     │
│  │  ├── 02_ChatIndex/       # Árvores de conversas      │
│  │  ├── 03_Docling_Output/  # Extrações estruturadas    │
│  │  ├── 04_Integracoes/     # Dados cruzados            │
│  │  └── 05_Auditoria/       # Logs unificados           │
│  └──────────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────────┘
"""


# Sistema Jurídico RAG Avançado (Notebook Demo)

Este notebook apresenta uma implementação de referência baseada na arquitetura de 4 camadas descrita na documentação técnica. Ele foca em:
- Extração estruturada (Docling)
- Indexação baseada em raciocínio (PageIndex)
- Gestão de contexto conversacional (ChatIndex)
- Persistência auditável (Google Drive)

Inclui também um chat jurídico robusto com rastreabilidade e logs de auditoria.


## Dependências e configuração

Este notebook foi estruturado para rodar localmente ou em ambientes como Colab. Ajuste os caminhos e chaves conforme necessário.

In [ ]:
# CÉLULA 1: INSTALAÇÃO E CONFIGURAÇÃO COMPLETA (execute apenas uma vez)
print("🚀 INICIALIZANDO SISTEMA JURÍDICO UNIFICADO AVANÇADO")
print("=" * 70)

print("\n📦 INSTALANDO DEPENDÊNCIAS...")
%pip install -qU langchain langchain-groq
%pip install -qU "docling[all]"
%pip install -qU "mcp[cli]" nest-asyncio
%pip install -qU beautifulsoup4 requests lxml html2text
%pip install -qU pymupdf pytesseract pillow
%pip install -qU aiohttp google-api-python-client google-auth
%pip install -qU scikit-learn networkx pandas numpy
%pip install -qU sentence-transformers
!apt-get update -q
!apt-get install -qy tesseract-ocr tesseract-ocr-por poppler-utils > /dev/null
!playwright install --with-deps chromium
print("✅ Pacotes instalados com sucesso!")


In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from datetime import datetime
import os
from pathlib import Path
from typing import Any, Dict, List, Optional
import asyncio
import requests
from docling.document_converter import DocumentConverter
from playwright.async_api import async_playwright
import hashlib
import json
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Configuração base (ajuste conforme seu ambiente)
@dataclass(frozen=True)
class ConfigSistema:
    pageindex_api_url: str = "https://api.pageindex.ai/v1/index"
    pageindex_api_key: str = "SUA_CHAVE_AQUI"
    chatindex_dir: Path = Path("./chatindex")
    drive_root: Path = Path("./Juridico_Unificado")
    audit_dir: Path = Path("./Juridico_Unificado/05_Auditoria")
    cache_dir: Path = Path("./cache_juridico")
    mcp_servers_dir: Path = Path("./Juridico_Unificado/04_Integracoes/mcp_servers")
    versao_fonte: str = "v1"
    groq_api_key: str = os.environ.get("GROQ_API_KEY", "")
    groq_model_analise: str = "llama-3.1-8b-instant"
    groq_model_resposta: str = "llama-3.1-70b-versatile"

config = ConfigSistema()
config

In [ ]:
# Configuração do ambiente e chaves (preencha antes de usar LLMs)
os.environ.setdefault("PAGEINDEX_API_KEY", "SUA_CHAVE_AQUI")
os.environ.setdefault("GROQ_API_KEY", "SUA_CHAVE_AQUI")

# Diretórios padrão (ajuste para /content/drive/... se estiver no Colab)
drive_root = Path("/content/drive/MyDrive/Juridico_Unificado")
drive_root.mkdir(parents=True, exist_ok=True)


In [ ]:
# ============================================
# CÉLULA 2: SISTEMA DE AUDITORIA E RASTREABILIDADE
# ============================================

print("\n" + "=" * 70)
print("📊 SISTEMA DE AUDITORIA E RASTREABILIDADE")
print("=" * 70)

import asyncio
import aiohttp
from typing import Dict, List, Any, Optional, Tuple, Set
import re
import numpy as np
import pandas as pd
from dataclasses import dataclass, field, asdict
from enum import Enum
import hashlib
import uuid
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta
from pathlib import Path
import json

CONFIG = {
    "audit_dir": str(config.audit_dir),
    "base_path": str(config.drive_root),
}

class EventoTipo(Enum):
    """Tipos de eventos auditáveis"""
    INICIO_PROCESSAMENTO = "inicio_processamento"
    DOWNLOAD_DOCUMENTO = "download_documento"
    EXTRACAO_DOCLING = "extracao_docling"
    INDEXACAO_PAGEINDEX = "indexacao_pageindex"
    CHUNKING_SEMANTICO = "chunking_semantico"
    ARMAZENAMENTO_CHATINDEX = "armazenamento_chatindex"
    INTEGRACAO_MCP = "integracao_mcp"
    CONSULTA_USUARIO = "consulta_usuario"
    RESPOSTA_GERADA = "resposta_gerada"
    ERRO_PROCESSAMENTO = "erro_processamento"
    BACKUP_DRIVE = "backup_drive"
    VERIFICACAO_INTEGRIDADE = "verificacao_integridade"

@dataclass
class EventoAuditoria:
    """Estrutura de evento auditável imutável"""
    id: str = field(default_factory=lambda: str(uuid.uuid4()))
    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())
    tipo: EventoTipo = EventoTipo.INICIO_PROCESSAMENTO
    componente: str = ""
    acao: str = ""
    detalhes: Dict[str, Any] = field(default_factory=dict)
    hash_anterior: str = ""
    hash_atual: str = ""
    arquivos_relacionados: List[str] = field(default_factory=list)
    metadata: Dict[str, Any] = field(default_factory=dict)

    def calcular_hash(self) -> str:
        """Calcula hash SHA-256 do evento"""
        dados = f"{self.timestamp}{self.tipo.value}{self.componente}{self.acao}{json.dumps(self.detalhes, sort_keys=True)}"
        return hashlib.sha256(dados.encode()).hexdigest()

class SistemaAuditoriaUnificado:
    """
    Sistema centralizado de auditoria com rastreabilidade completa
    Implementa cadeia de hashes imutável
    """

    def __init__(self, config: Dict[str, Any]):
        self.config = config
        self.audit_dir = Path(config["audit_dir"])
        self.log_central = []
        self.hash_cadeia = ""

        # Inicializar estrutura de auditoria
        self._inicializar_auditoria()

    def _inicializar_auditoria(self):
        """Inicializa sistema de auditoria"""
        # Criar estrutura de logs
        (self.audit_dir / "logs_diarios").mkdir(exist_ok=True)
        (self.audit_dir / "backups_hash").mkdir(exist_ok=True)
        (self.audit_dir / "relatorios").mkdir(exist_ok=True)

        # Arquivo de log principal (append-only)
        self.log_principal = self.audit_dir / "auditoria_principal.jsonl"

        # Arquivo de índices para busca rápida
        self.indice_eventos = self.audit_dir / "indice_eventos.json"

        # Inicializar cadeia de hashes
        evento_genese = EventoAuditoria(
            tipo=EventoTipo.INICIO_PROCESSAMENTO,
            componente="SistemaAuditoria",
            acao="Inicialização do sistema",
            detalhes={"versao": "1.0.0", "configuracao": self.config},
            hash_anterior="0" * 64,
        )
        evento_genese.hash_atual = evento_genese.calcular_hash()
        self.hash_cadeia = evento_genese.hash_atual

        # Registrar evento de gênese
        self._registrar_evento_persistente(evento_genese)

        print(f"✅ Auditoria inicializada | Hash inicial: {self.hash_cadeia[:16]}...")

    def registrar_evento(self, evento: EventoAuditoria) -> str:
        """
        Registra evento com cadeia de hashes imutável
        """
        # Vincular ao hash anterior
        evento.hash_anterior = self.hash_cadeia

        # Calcular hash atual
        evento.hash_atual = evento.calcular_hash()

        # Atualizar cadeia
        self.hash_cadeia = evento.hash_atual

        # Persistir evento
        self._registrar_evento_persistente(evento)

        # Atualizar índice
        self._atualizar_indice(evento)

        # Backup periódico
        if len(self.log_central) % 100 == 0:
            self._criar_backup_hash()

        return evento.id

    def _registrar_evento_persistente(self, evento: EventoAuditoria):
        """Persiste evento em múltiplos formatos"""

        # 1. Log principal (JSONL)
        with open(self.log_principal, "a", encoding="utf-8") as f:
            f.write(json.dumps(asdict(evento), ensure_ascii=False, default=str) + "\n")

        # 2. Log diário separado por data
        data_str = evento.timestamp[:10]  # YYYY-MM-DD
        log_diario = self.audit_dir / "logs_diarios" / f"{data_str}.jsonl"
        with open(log_diario, "a", encoding="utf-8") as f:
            f.write(json.dumps(asdict(evento), ensure_ascii=False, default=str) + "\n")

        # 3. Armazenar em memória (limitado)
        self.log_central.append(evento)
        if len(self.log_central) > 1000:
            self.log_central = self.log_central[-1000:]

    def _atualizar_indice(self, evento: EventoAuditoria):
        """Atualiza índice de eventos para busca rápida"""

        indice_path = self.indice_eventos
        indice = {}

        if indice_path.exists():
            try:
                indice = json.loads(indice_path.read_text())
            except:
                indice = {}

        # Adicionar ao índice
        if evento.tipo.value not in indice:
            indice[evento.tipo.value] = []

        entrada_indice = {
            "id": evento.id,
            "timestamp": evento.timestamp,
            "componente": evento.componente,
            "hash": evento.hash_atual[:16],
        }

        indice[evento.tipo.value].append(entrada_indice)

        # Manter índice limitado
        for tipo in indice:
            indice[tipo] = indice[tipo][-1000:]

        # Salvar índice
        indice_path.write_text(json.dumps(indice, indent=2, ensure_ascii=False, default=str))

    def _criar_backup_hash(self):
        """Cria backup da cadeia de hashes"""
        backup_file = self.audit_dir / "backups_hash" / f"backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

        backup_data = {
            "timestamp": datetime.now().isoformat(),
            "hash_cadeia": self.hash_cadeia,
            "total_eventos": len(self.log_central),
            "ultimos_10_hashes": [e.hash_atual[:16] for e in self.log_central[-10:]],
        }

        backup_file.write_text(json.dumps(backup_data, indent=2, ensure_ascii=False, default=str))

    def gerar_relatorio_rastreabilidade(self, evento_id: str = None, consulta_id: str = None) -> Dict[str, Any]:
        """
        Gera relatório completo de rastreabilidade
        """

        relatorio = {
            "metadata": {
                "gerado_em": datetime.now().isoformat(),
                "sistema": "Juridico_Unificado",
                "hash_cadeia_atual": self.hash_cadeia,
            },
            "cadeia_eventos": [],
            "integridade_verificada": True,
            "arquivos_relacionados": [],
            "timeline_detalhada": [],
        }

        # Buscar eventos relacionados
        eventos_relacionados = []

        if evento_id:
            # Buscar por ID específico
            for evento in self.log_central:
                if evento.id == evento_id:
                    eventos_relacionados.append(evento)
                    break

        if consulta_id:
            # Buscar todos eventos de uma consulta
            for evento in self.log_central:
                if "consulta_id" in evento.detalhes and evento.detalhes["consulta_id"] == consulta_id:
                    eventos_relacionados.append(evento)

        # Reconstruir cadeia
        if eventos_relacionados:
            evento_inicial = eventos_relacionados[0]
            hash_atual = evento_inicial.hash_anterior

            # Buscar cadeia completa
            while hash_atual != "0" * 64:
                for evento in reversed(self.log_central):
                    if evento.hash_atual == hash_atual:
                        relatorio["cadeia_eventos"].append(asdict(evento))
                        hash_atual = evento.hash_anterior
                        break
                else:
                    break

            # Adicionar eventos relacionados
            relatorio["cadeia_eventos"].extend([asdict(e) for e in eventos_relacionados])

            # Coletar arquivos
            for evento in eventos_relacionados:
                relatorio["arquivos_relacionados"].extend(evento.arquivos_relacionados)

            # Timeline detalhada
            eventos_ordenados = sorted(eventos_relacionados, key=lambda x: x.timestamp)
            for evento in eventos_ordenados:
                relatorio["timeline_detalhada"].append({
                    "timestamp": evento.timestamp,
                    "tipo": evento.tipo.value,
                    "acao": evento.acao,
                    "duracao": evento.detalhes.get("duracao_ms", 0),
                })

        # Verificar integridade da cadeia
        relatorio["integridade_verificada"] = self._verificar_integridade_cadeia()

        # Salvar relatório
        relatorio_path = self.audit_dir / "relatorios" / f"rastreabilidade_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        relatorio_path.write_text(json.dumps(relatorio, indent=2, ensure_ascii=False, default=str))

        return relatorio

    def _verificar_integridade_cadeia(self) -> bool:
        """Verifica integridade da cadeia de hashes"""

        if len(self.log_central) < 2:
            return True

        for i in range(1, len(self.log_central)):
            evento_atual = self.log_central[i]
            evento_anterior = self.log_central[i - 1]

            if evento_atual.hash_anterior != evento_anterior.hash_atual:
                print(f"⚠️  Violação de integridade na posição {i}")
                print(f"   Hash anterior esperado: {evento_anterior.hash_atual[:16]}")
                print(f"   Hash anterior encontrado: {evento_atual.hash_anterior[:16]}")
                return False

        return True

    def monitorar_performance(self, periodo_horas: int = 24) -> Dict[str, Any]:
        """Monitora performance do sistema"""

        cutoff_time = datetime.now() - timedelta(hours=periodo_horas)

        eventos_periodo = [
            e for e in self.log_central
            if datetime.fromisoformat(e.timestamp.replace("Z", "+00:00")) > cutoff_time
        ]

        metricas = {
            "total_eventos": len(eventos_periodo),
            "por_tipo": {},
            "tempo_medio_resposta": 0,
            "taxa_erros": 0,
            "componentes_ativos": set(),
        }

        tempos_resposta = []
        erros = 0

        for evento in eventos_periodo:
            # Contagem por tipo
            tipo = evento.tipo.value
            metricas["por_tipo"][tipo] = metricas["por_tipo"].get(tipo, 0) + 1

            # Coletar componentes
            metricas["componentes_ativos"].add(evento.componente)

            # Tempos de resposta
            if "duracao_ms" in evento.detalhes:
                tempos_resposta.append(evento.detalhes["duracao_ms"])

            # Contar erros
            if evento.tipo == EventoTipo.ERRO_PROCESSAMENTO:
                erros += 1

        # Calcular métricas
        if tempos_resposta:
            metricas["tempo_medio_resposta"] = sum(tempos_resposta) / len(tempos_resposta)

        if eventos_periodo:
            metricas["taxa_erros"] = erros / len(eventos_periodo)

        metricas["componentes_ativos"] = list(metricas["componentes_ativos"])

        return metricas

# Inicializar sistema de auditoria
auditoria = SistemaAuditoriaUnificado(CONFIG)

print("✅ Sistema de auditoria inicializado com cadeia de hashes imutável")


## 🎯 Como usar o sistema

1. **Processar documentos:**
   ```python
   resultado = sistema.processar_documento(Path("seu_documento.pdf"))
   ```
2. **Consultar o sistema:**
   ```python
   resposta = sistema.responder_consulta("sua pergunta jurídica", {"fontes": []})
   ```
3. **Gerar relatórios:**
   ```python
   relatorio = sistema.auditoria.registrar_evento("relatorios", {"tipo": "snapshot"})
   ```
4. **Acessar auditoria:**
   ```python
   sistema.auditoria.registrar_evento("consultas", {"tipo": "consulta", "consulta": "exemplo"})
   ```

### 🔧 Configurações importantes
- **API Keys:** configure `GROQ_API_KEY` para respostas com LLM.
- **Fontes:** scraping inclui STF, Planalto e STJ.
- **Armazenamento:** tudo é salvo no Google Drive de forma estruturada.
- **Chunking:** preserva estrutura jurídica e evita quebras inadequadas.

### 🚀 Próximos passos
1. Configurar API do PageIndex para indexação em árvore real.
2. Implementar servidores MCP locais para melhor integração.
3. Adicionar mais fontes oficiais de scraping.
4. Implementar cache distribuído para melhor performance.
5. Adicionar dashboard de monitoramento.

### 📁 Estrutura no Google Drive
```text
Juridico_Unificado/
├── 01_PageIndex/          # Árvores de documentos
├── 02_ChatIndex/         # Histórico de conversas
├── 03_Docling_Output/    # Extrações estruturadas
│   ├── raw_extractions/  # Extrações brutas
│   ├── structured_outputs/ # Extrações processadas
│   └── chunks_semanticos/ # Chunks preservados
├── 04_Integracoes/       # Dados cruzados
├── 05_Auditoria/         # Logs e rastreabilidade
│   ├── log_central.jsonl # Log principal
│   ├── relatorios/       # Relatórios gerados
│   └── consultas/        # Histórico de consultas
└── indice_documentos.json # Índice global
```

### ⚠️ Notas importantes
- O scraping de fontes reais requer ajustes para sites específicos.
- A integração completa com PageIndex MCP requer servidor em execução.
- Configure `GROQ_API_KEY` para respostas mais inteligentes.
- Documentos muito grandes podem exigir ajustes no chunking.

✅ O sistema está pronto para processar documentos jurídicos com:
- Extração estrutural com Docling
- Chunking semântico que preserva contexto
- Scraping de fontes oficiais
- Armazenamento rastreável no Google Drive
- Auditoria completa de todas as operações


In [ ]:
# Exemplo: baixar PDF para processamento
def baixar_pdf(url: str, destino: Path) -> Path:
    destino.parent.mkdir(parents=True, exist_ok=True)
    resposta = requests.get(url, timeout=30)
    resposta.raise_for_status()
    destino.write_bytes(resposta.content)
    return destino

pdf_url = "https://arxiv.org/pdf/2103.15348.pdf"
pdf_path = baixar_pdf(pdf_url, Path("./cache_juridico/exemplo.pdf"))
print(f"PDF salvo em: {pdf_path}")


## Camada 1: Orquestração

A classe `SistemaJuridicoUnificado` atua como facade, coordenando extração, chunking, indexação, scraping e auditoria.

### Integrações reais (Docling, PageIndex, scraping)

Os componentes abaixo conectam o fluxo a extração real via Docling, indexação na API do PageIndex e scraping com Playwright, incluindo rate limiting.


In [ ]:
class SistemaExtracaoDocling:
    def __init__(self) -> None:
        self.converter = DocumentConverter()

    def extrair(self, documento_path: Path) -> Dict[str, Any]:
        resultado = self.converter.convert(str(documento_path))
        documento = resultado.document
        texto = documento.export_to_text()
        return {
            "documento_id": documento_path.stem,
            "texto_completo": texto,
            "metadata": getattr(resultado, "metadata", {}),
        }


class PageIndexClient:
    def __init__(self, api_url: str, api_key: str) -> None:
        self.api_url = api_url
        self.api_key = api_key

    def indexar(self, documento_id: str, chunks: List[str]) -> Dict[str, Any]:
        resposta = requests.post(
            self.api_url,
            json={"documento_id": documento_id, "chunks": chunks},
            headers={"Authorization": f"Bearer {self.api_key}"},
            timeout=30,
        )
        resposta.raise_for_status()
        return resposta.json()


class SistemaDownload:
    def __init__(self, versao_fonte: str) -> None:
        self.versao_fonte = versao_fonte

    def registrar_fonte(self, origem: str) -> Dict[str, Any]:
        return {
            "origem": origem,
            "versao_fonte": self.versao_fonte,
            "data_captura": datetime.now().isoformat(),
        }


class SistemaScrapingJuridico:
    def __init__(self, downloader: SistemaDownload, rate_limit_seconds: float = 1.0) -> None:
        self.rate_limit_seconds = rate_limit_seconds
        self.downloader = downloader

    async def coletar_texto(self, url: str) -> Dict[str, Any]:
        async with async_playwright() as playwright:
            browser = await playwright.chromium.launch()
            page = await browser.new_page()
            await page.goto(url, wait_until="networkidle")
            await asyncio.sleep(self.rate_limit_seconds)
            texto = await page.inner_text("body")
            await browser.close()
            return {"texto": texto, "metadata": self.downloader.registrar_fonte(url)}


class ChatIndexPersistente:
    def __init__(self, storage_path: Path) -> None:
        self.storage_path = storage_path
        self.storage_path.parent.mkdir(parents=True, exist_ok=True)

    def registrar(self, conversa: Dict[str, Any]) -> None:
        with self.storage_path.open("a", encoding="utf-8") as handle:
            handle.write(json.dumps(conversa, ensure_ascii=False) + "\n")

    def carregar(self) -> List[Dict[str, Any]]:
        if not self.storage_path.exists():
            return []
        return [
            json.loads(linha)
            for linha in self.storage_path.read_text(encoding="utf-8").splitlines()
            if linha.strip()
        ]


In [ ]:
# Exemplo: scraping simples de uma página
sistema = SistemaJuridicoUnificado(config)
texto_coletado = asyncio.run(sistema.scraper.coletar_texto("https://www.gov.br/"))
print(texto_coletado["metadata"])


In [ ]:
class SistemaAuditoriaUnificado:
    def __init__(self, audit_dir: Path):
        self.audit_dir = audit_dir
        self.audit_dir.mkdir(parents=True, exist_ok=True)
        self.log_central: List[Dict[str, Any]] = []
        self.hash_registry: Dict[str, Dict[str, str]] = {}

    def registrar_evento(self, categoria: str, evento: Dict[str, Any]) -> str:
        evento_id = f"evt_{hashlib.md5(str(evento).encode()).hexdigest()[:10]}"
        evento["timestamp"] = evento.get("timestamp", datetime.now().isoformat())

        if self.log_central:
            evento["hash_anterior"] = self.hash_registry[self.log_central[-1]["evento_id"]]["hash"]

        hash_atual = hashlib.md5(json.dumps(evento, sort_keys=True).encode()).hexdigest()
        self.hash_registry[evento_id] = {"hash": hash_atual, "timestamp": evento["timestamp"]}

        registro = {**evento, "evento_id": evento_id, "hash_atual": hash_atual}
        self.log_central.append(registro)
        self._persistir_log(categoria, registro)
        return evento_id

    def _persistir_log(self, categoria: str, registro: Dict[str, Any]) -> None:
        destino = self.audit_dir / f"{categoria}.jsonl"
        with destino.open("a", encoding="utf-8") as handle:
            handle.write(json.dumps(registro, ensure_ascii=False) + "\n")


class SistemaJuridicoUnificado:
    def __init__(self, config: ConfigSistema):
        self.config = config
        self.auditoria = SistemaAuditoriaUnificado(config.audit_dir)
        self.extrator = SistemaExtracaoDocling()
        self.pageindex = PageIndexClient(config.pageindex_api_url, config.pageindex_api_key)
        self.downloader = SistemaDownload(config.versao_fonte)
        self.scraper = SistemaScrapingJuridico(self.downloader)
        self.chatindex = ChatIndexPersistente(config.chatindex_dir / "historico.jsonl")

    def _chunking_semantico(self, texto: str, tamanho: int = 800) -> List[str]:
        return [texto[i:i + tamanho] for i in range(0, len(texto), tamanho)]

    def _extrair_versoes_fontes(self, metadados_fontes: List[Dict[str, Any]]) -> List[str]:
    def _gerar_resposta_llm(self, consulta: str, fontes: List[str], metadados_fontes: List[Dict[str, Any]]) -> str:
        if not self.config.groq_api_key or self.config.groq_api_key == "SUA_CHAVE_AQUI":
            return ""
        analise_prompt = ChatPromptTemplate.from_messages([
            ("system", "Você é um analista jurídico. Extraia pontos-chave e contexto necessário."),
            ("human", "Consulta: {consulta}\nFontes: {fontes}")
        ])
        resposta_prompt = ChatPromptTemplate.from_messages([
            ("system", "Você é um assistente jurídico. Responda com base nos pontos-chave e fontes."),
            ("human", "Consulta: {consulta}\nPontos-chave: {pontos}\nMetadados: {metadados}")
        ])
        llm_analise = ChatGroq(
            groq_api_key=self.config.groq_api_key,
            model=self.config.groq_model_analise,
            temperature=0.2,
        )
        llm_resposta = ChatGroq(
            groq_api_key=self.config.groq_api_key,
            model=self.config.groq_model_resposta,
            temperature=0.3,
        )
        pontos = (analise_prompt | llm_analise | StrOutputParser()).invoke({
            "consulta": consulta,
            "fontes": ", ".join(fontes),
        })
        return (resposta_prompt | llm_resposta | StrOutputParser()).invoke({
            "consulta": consulta,
            "pontos": pontos,
            "metadados": metadados_fontes,
        })

        return sorted({
            item.get("versao_fonte")
            for item in metadados_fontes
            if item.get("versao_fonte")
        })

    def processar_documento(self, documento_path: Path) -> Dict[str, Any]:
        self.auditoria.registrar_evento("processamento", {
            "tipo": "inicio_processamento",
            "documento": documento_path.name,
        })
        extracao = self.extrator.extrair(documento_path)
        chunks = self._chunking_semantico(extracao["texto_completo"])
        indexacao = self.pageindex.indexar(extracao["documento_id"], chunks)
        arvore = {"documento_id": documento_path.stem, "estrutura_arvore": {"raiz": {"titulo": documento_path.stem}}}

        self.auditoria.registrar_evento("processamento", {
            "tipo": "fim_processamento",
            "documento": documento_path.name,
            "chunks_gerados": len(chunks),
        })
        return {"extracao": extracao, "chunks": chunks, "arvore": arvore, "indexacao": indexacao}

    def responder_consulta(self, consulta: str, contexto: Dict[str, Any]) -> Dict[str, Any]:
        metadados_fontes = contexto.get("metadados_fontes", [])
        versoes_fontes = self._extrair_versoes_fontes(metadados_fontes)
        self.auditoria.registrar_evento("consultas", {
            "tipo": "consulta",
            "consulta": consulta,
            "versao_fonte": versoes_fontes,
        })
        resposta_texto = self._gerar_resposta_llm(consulta, contexto.get("fontes", []), metadados_fontes)
        resposta = {
            "consulta": consulta,
            "resposta": resposta_texto or f"Resumo jurídico para: {consulta}",
            "fontes": contexto.get("fontes", []),
            "metadados_fontes": metadados_fontes,
            "versao_fonte": versoes_fontes,
        }
        self.auditoria.registrar_evento("consultas", {
            "tipo": "resposta",
            "consulta": consulta,
            "hash_resposta": hashlib.md5(json.dumps(resposta, ensure_ascii=False).encode()).hexdigest(),
            "versao_fonte": versoes_fontes,
        })
        return resposta


## Camada 2: Busca híbrida e ranking

A classe `SistemaBuscaHibrida` combina sinais do PageIndex e de embeddings vetoriais, com pesos configuráveis e explicabilidade do ranking.


In [ ]:
class SistemaBuscaHibrida:
    def __init__(self, peso_pageindex: float = 0.6, peso_vetorial: float = 0.4):
        self.peso_pageindex = peso_pageindex
        self.peso_vetorial = peso_vetorial

    def _normalizar(self, resultados: list[dict[str, float]]) -> dict[str, float]:
        if not resultados:
            return {}
        max_score = max(item.get("score", 0.0) for item in resultados) or 1.0
        return {item["id"]: item.get("score", 0.0) / max_score for item in resultados}

    def rankear(self, resultados_pageindex: list[dict[str, float]], resultados_vetoriais: list[dict[str, float]]):
        pageindex_norm = self._normalizar(resultados_pageindex)
        vetorial_norm = self._normalizar(resultados_vetoriais)
        ids = set(pageindex_norm) | set(vetorial_norm)
        combinados = []

        for doc_id in ids:
            score_pageindex = pageindex_norm.get(doc_id, 0.0)
            score_vetorial = vetorial_norm.get(doc_id, 0.0)
            score_final = (
                self.peso_pageindex * score_pageindex +
                self.peso_vetorial * score_vetorial
            )
            combinados.append({
                "id": doc_id,
                "score_final": score_final,
                "score_pageindex": score_pageindex,
                "score_vetorial": score_vetorial,
            })

        combinados.sort(key=lambda item: item["score_final"], reverse=True)
        return {
            "resultados": combinados,
            "explicabilidade": {
                "criterios": (
                    "Score final = peso_pageindex * score_pageindex_normalizado + peso_vetorial * score_vetorial_normalizado"
                ),
                "pesos": {
                    "peso_pageindex": self.peso_pageindex,
                    "peso_vetorial": self.peso_vetorial,
                },
            },
        }


busca_hibrida = SistemaBuscaHibrida(peso_pageindex=0.7, peso_vetorial=0.3)
rankeamento = busca_hibrida.rankear(
    resultados_pageindex=[{"id": "doc_1", "score": 0.9}, {"id": "doc_2", "score": 0.7}],
    resultados_vetoriais=[{"id": "doc_2", "score": 0.95}, {"id": "doc_3", "score": 0.6}],
)
rankeamento


## Chat Jurídico Robusto

Este fluxo simula um chat jurídico que:
- Mantém histórico de contexto
- Registra auditoria de cada interação
- Retorna respostas com fontes rastreáveis


In [ ]:
class ChatJuridico:
    def __init__(self, sistema: SistemaJuridicoUnificado):
        self.sistema = sistema
        self.historico: List[Dict[str, Any]] = []

    def enviar(self, consulta: str, fontes: Optional[List[str]] = None) -> Dict[str, Any]:
        fontes_list = fontes or []
        metadados_fontes = [
            self.sistema.downloader.registrar_fonte(fonte)
            for fonte in fontes_list
        ]
        contexto = {"fontes": fontes_list, "metadados_fontes": metadados_fontes}
        resposta = self.sistema.responder_consulta(consulta, contexto)
        self.historico.append({"consulta": consulta, "resposta": resposta})
        self.sistema.chatindex.registrar({"consulta": consulta, "resposta": resposta})
        return resposta


sistema = SistemaJuridicoUnificado(config)
chat = ChatJuridico(sistema)

chat.enviar("Quais são os requisitos para tutela de urgência?", ["STJ", "Planalto"])

In [ ]:
# Conversação de exemplo com o chat jurídico
sistema = SistemaJuridicoUnificado(config)
chat = ChatJuridico(sistema)

chat.enviar("Quais são os requisitos para tutela de urgência?", ["STJ", "Planalto"])
chat.enviar("Qual a base legal da responsabilidade civil objetiva?", ["STF", "Planalto"])


## Avaliação e testes de regressão

Exemplos simples de métricas para validar a qualidade do ranking e detectar regressões.


In [ ]:
def recall_at_k(resultados: List[str], relevantes: List[str], k: int = 5) -> float:
    resultados_k = set(resultados[:k])
    relevantes_set = set(relevantes)
    if not relevantes_set:
        return 0.0
    return len(resultados_k & relevantes_set) / len(relevantes_set)


def avaliacao_regressao(resultados: List[str], relevantes: List[str], limite: float = 0.6) -> Dict[str, Any]:
    recall = recall_at_k(resultados, relevantes, k=5)
    aprovado = recall >= limite
    return {"recall@5": recall, "limite": limite, "aprovado": aprovado}


avaliacao_regressao(["doc_1", "doc_2", "doc_3"], ["doc_2", "doc_4"])


## Próximos passos

1. Substituir os placeholders por integrações reais (Docling, PageIndex API).
2. Implementar scraping com Playwright e rate limiting.
3. Enriquecer o ChatIndex com histórico persistente.
4. Adicionar testes de regressão e avaliação de qualidade.
